In [1]:
from sqlalchemy import create_engine
import pandas as pd
import yaml
import numpy as np
import sklearn
import os
from os.path import isfile

In [2]:
fb_df = pd.read_json("data/products_table.json")

In [ ]:
fb_df.shape
#8091 rows and 8 columns

In [3]:
def remove_n_a_records(df, column: str):
    """
    Scan the column for records with all N/As. Get rid of them

    Args:
        column (str): The column currently being scanned.
    """
    # Swap N/A for the pandas nan, so we can drop them
    temp_df = df[column].replace('N/A', np.nan)
    temp_df = temp_df.dropna()
    # Create a new df with only the records without the nans
    clean_df = pd.merge(temp_df, df,
                        left_index=True, right_index=True)
    # The merge creates a duplicate column. Remove it.
    clean_df.drop(column + '_x', inplace=True, axis=1)
    # Rename the remaining category column
    clean_df.rename(columns={column + '_y': column}, inplace=True)
    # Commit the cleansed data to the dataframe
    df = clean_df
    return df

fb_df = remove_n_a_records(fb_df, 'category')

In [4]:
fb_df

,id,product_name,category,product_description,price,location,page_id,create_time
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",1426704584,2022-02-26
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",1426704579,2022-02-26
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",1426704576,2022-02-26
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",1426704575,2022-02-26
5,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,£450.00,"Delph, Manchester",1426704570,2022-02-26
...,...,...,...,...,...,...,...,...
8085,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,£260.00,"Acocks Green, West Midlands",1422159237,2022-02-28
8086,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,£235.00,"Acocks Green, West Midlands",1422159464,2022-02-28
8088,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,£250.00,"Montrose, Angus",1426668818,2022-02-28
8089,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,£30.00,"Carrickfergus, County Antrim",1426699715,2022-02-28


In [ ]:
#seperate numerical columns and select numerical columns
df_numeric = fb_df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

In [ ]:
#select non-numeric columns
df_non_numeric = fb_df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values

In [5]:
fb_df['price'] = fb_df['price'].apply(
            lambda x: x.strip("£").replace(',',''))
fb_df['price'] = fb_df['price'].astype('float64')

In [6]:
fb_df['price'].head(10)

1       5.0
2      20.0
3      20.0
4     115.0
5     450.0
6       9.0
7     419.0
8      10.0
9      50.0
10    200.0
Name: price, dtype: float64

In [7]:
fb_df

,id,product_name,category,product_description,price,location,page_id,create_time
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",5.0,"Wokingham, Berkshire",1426704584,2022-02-26
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,20.0,"Inverness, Highland",1426704579,2022-02-26
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,20.0,"Skegness, Lincolnshire",1426704576,2022-02-26
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,115.0,"Radstock, Somerset",1426704575,2022-02-26
5,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,450.0,"Delph, Manchester",1426704570,2022-02-26
...,...,...,...,...,...,...,...,...
8085,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,260.0,"Acocks Green, West Midlands",1422159237,2022-02-28
8086,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,235.0,"Acocks Green, West Midlands",1422159464,2022-02-28
8088,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,250.0,"Montrose, Angus",1426668818,2022-02-28
8089,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,30.0,"Carrickfergus, County Antrim",1426699715,2022-02-28


In [ ]:
#check for outliers
fb_df['price'].describe()

In [ ]:
fb_df['price'].plot(kind='box', figsize=(12, 8))

In [8]:
def remove_price_outliers(df):
    
    df = df[df['price'] < 10000]
    df = df[df['price'] > 0.1]
    return df

fb_df = remove_price_outliers(fb_df)

In [ ]:
fb_df['price'].plot(kind='box', figsize=(12, 8))

In [ ]:
#check for outliers
fb_df['price'].describe()

In [ ]:
#fixing data type
fb_df['create_time'].dtype

In [ ]:
#convert create_time to datetime format
fb_df['create_time'] = pd.to_datetime(fb_df.create_time, format='%Y-%m-%d')

In [ ]:
fb_df.info()

In [ ]:
fb_df

In [9]:
#split categories into main using lambda
fb_df['main_category'] = fb_df['category'].apply(
    lambda x: x.split("/")[0].split())


In [10]:
fb_df

,id,product_name,category,product_description,price,location,page_id,create_time,main_category
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",5.0,"Wokingham, Berkshire",1426704584,2022-02-26,"[Home, &, Garden]"
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,20.0,"Inverness, Highland",1426704579,2022-02-26,"[Home, &, Garden]"
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,20.0,"Skegness, Lincolnshire",1426704576,2022-02-26,"[Home, &, Garden]"
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,115.0,"Radstock, Somerset",1426704575,2022-02-26,"[Home, &, Garden]"
5,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,450.0,"Delph, Manchester",1426704570,2022-02-26,"[Home, &, Garden]"
...,...,...,...,...,...,...,...,...,...
8085,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,260.0,"Acocks Green, West Midlands",1422159237,2022-02-28,"[Video, Games, &, Consoles]"
8086,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,235.0,"Acocks Green, West Midlands",1422159464,2022-02-28,"[Video, Games, &, Consoles]"
8088,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,250.0,"Montrose, Angus",1426668818,2022-02-28,"[Video, Games, &, Consoles]"
8089,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,30.0,"Carrickfergus, County Antrim",1426699715,2022-02-28,"[Video, Games, &, Consoles]"


In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(x='category', y='price', data=fb_df)

In [ ]:
fb_df

In [12]:
#create model and drop price column

product_X = fb_df['category']
product_y = fb_df['price']
product_X = pd.get_dummies(product_X, drop_first=True)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

product_X_train, product_X_test, product_y_train, product_y_test = train_test_split(product_X, product_y, test_size=0.2, random_state=1)

regr = LinearRegression()
regr.fit(product_X_train, product_y_train)

product_y_pred = regr.predict(product_X_test)
product_y_pred



array([271.95398803, 253.20789428, 118.93445678, ...,   3.00086303,
        19.63928191,  90.87195678])